In [31]:
!pip install -Uqq fastai

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
from fastai.vision.all import *
from pathlib import Path

# Cargar los datos desde los archivos CSV
train_df = pd.read_csv("/content/drive/MyDrive/Nuwe/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Nuwe/test_no_label.csv")


base_path = Path("/content/drive/MyDrive/Nuwe/")

train_df['path_img'] = train_df['path_img'].apply(lambda x: base_path / x)
test_df['path_img'] = test_df['path_img'].apply(lambda x: base_path / x)


In [61]:
test_df

,idx_test,path_img,predicted_label
0,0,/content/drive/MyDrive/Nuwe/all_imgs/02191c3b-1707-4053-9aa6-f7ada55780ef.jpg,1
1,1,/content/drive/MyDrive/Nuwe/all_imgs/e6fa45d5-78ee-4799-8f1d-195671b56cba.jpg,1
2,2,/content/drive/MyDrive/Nuwe/all_imgs/a190bf53-f9c8-4f10-a1e5-25aca7f23aaf.jpg,1
3,3,/content/drive/MyDrive/Nuwe/all_imgs/60dbc020-2733-4189-8899-ccd5d3c482d8.jpg,2
4,4,/content/drive/MyDrive/Nuwe/all_imgs/366f3884-ac1a-4e2f-8a5a-6e7ce5942085.jpg,1
...,...,...,...
122,122,/content/drive/MyDrive/Nuwe/all_imgs/800cb28f-545b-4d55-bbd7-2cd6bf1949ce.jpg,1
123,123,/content/drive/MyDrive/Nuwe/all_imgs/3c3d32d7-6dd5-427e-875d-53de71a9d92a.jpg,1
124,124,/content/drive/MyDrive/Nuwe/all_imgs/4d49f4a6-c0bf-477a-93b1-3c721b0a5cf9.jpg,1
125,125,/content/drive/MyDrive/Nuwe/all_imgs/d998706b-9a45-415e-8a95-8166a8db0b02.jpg,1


In [35]:
# Crear un DataBlock para cargar los datos
blocks = (ImageBlock, CategoryBlock)
getters = [ColReader('path_img'), ColReader('label', label_delim=None)]

flower_data = DataBlock(blocks=blocks,
                        getters=getters,
                        splitter=RandomSplitter(seed=42))

In [36]:
# Cargar los datos
dls = flower_data.dataloaders(train_df, bs=64)


In [37]:
# Utilizar ResNet50 pre-entrenado como modelo base
learn = vision_learner(dls, resnet50, metrics=accuracy)

In [46]:
# Entrenar el modelo
learn.fine_tune(epochs=15)

epoch,train_loss,valid_loss,accuracy,time
0,0.059025,2.002397,0.637931,00:02


epoch,train_loss,valid_loss,accuracy,time
0,0.059127,2.019043,0.603448,00:03
1,0.067319,1.932618,0.603448,00:03
2,0.084203,1.964231,0.586207,00:03
3,0.084119,2.124299,0.500000,00:03
4,0.086434,2.160796,0.517241,00:04
5,0.091245,2.032523,0.465517,00:04
6,0.092263,1.959998,0.517241,00:04
7,0.084532,1.807100,0.620690,00:03
8,0.079402,1.696921,0.672414,00:03
9,0.080969,1.654914,0.672414,00:03


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c4c7938b520>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7c4c7938b520>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
        self._shutdown_workers()
self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():    
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
if w.is_alive():
      File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
assert self._parent_pid == os.getpid(), 'can only test a

In [47]:
# Hacer predicciones en el conjunto de prueba
test_dl = dls.test_dl(test_df)
preds, _ = learn.get_preds(dl=test_dl)

In [58]:
# Mostrar las predicciones
predicted_labels = [learn.dls.vocab[i] for i in preds.argmax(dim=-1).numpy()]
test_df['predicted_label'] = predicted_labels
print(test_df)

     idx_test  \
0           0   
1           1   
2           2   
3           3   
4           4   
..        ...   
122       122   
123       123   
124       124   
125       125   
126       126   

                                                                          path_img  \
0    /content/drive/MyDrive/Nuwe/all_imgs/02191c3b-1707-4053-9aa6-f7ada55780ef.jpg   
1    /content/drive/MyDrive/Nuwe/all_imgs/e6fa45d5-78ee-4799-8f1d-195671b56cba.jpg   
2    /content/drive/MyDrive/Nuwe/all_imgs/a190bf53-f9c8-4f10-a1e5-25aca7f23aaf.jpg   
3    /content/drive/MyDrive/Nuwe/all_imgs/60dbc020-2733-4189-8899-ccd5d3c482d8.jpg   
4    /content/drive/MyDrive/Nuwe/all_imgs/366f3884-ac1a-4e2f-8a5a-6e7ce5942085.jpg   
..                                                                             ...   
122  /content/drive/MyDrive/Nuwe/all_imgs/800cb28f-545b-4d55-bbd7-2cd6bf1949ce.jpg   
123  /content/drive/MyDrive/Nuwe/all_imgs/3c3d32d7-6dd5-427e-875d-53de71a9d92a.jpg   
124  /content/drive/M

In [49]:
test_df.to_csv("predictions.csv")

In [60]:
pred = pd.read_csv("predictions.csv")
pred

,Unnamed: 0,idx_test,path_img,predicted_label
0,0,0,/content/drive/MyDrive/Nuwe/all_imgs/02191c3b-1707-4053-9aa6-f7ada55780ef.jpg,1
1,1,1,/content/drive/MyDrive/Nuwe/all_imgs/e6fa45d5-78ee-4799-8f1d-195671b56cba.jpg,1
2,2,2,/content/drive/MyDrive/Nuwe/all_imgs/a190bf53-f9c8-4f10-a1e5-25aca7f23aaf.jpg,1
3,3,3,/content/drive/MyDrive/Nuwe/all_imgs/60dbc020-2733-4189-8899-ccd5d3c482d8.jpg,2
4,4,4,/content/drive/MyDrive/Nuwe/all_imgs/366f3884-ac1a-4e2f-8a5a-6e7ce5942085.jpg,1
...,...,...,...,...
122,122,122,/content/drive/MyDrive/Nuwe/all_imgs/800cb28f-545b-4d55-bbd7-2cd6bf1949ce.jpg,1
123,123,123,/content/drive/MyDrive/Nuwe/all_imgs/3c3d32d7-6dd5-427e-875d-53de71a9d92a.jpg,1
124,124,124,/content/drive/MyDrive/Nuwe/all_imgs/4d49f4a6-c0bf-477a-93b1-3c721b0a5cf9.jpg,1
125,125,125,/content/drive/MyDrive/Nuwe/all_imgs/d998706b-9a45-415e-8a95-8166a8db0b02.jpg,1


In [73]:
newdf = pred[['idx_test', 'predicted_label']].copy()
newdf['predicted_label'] = newdf['predicted_label'].apply(int)


In [78]:
# Crear un diccionario con las predicciones en el formato deseado
predictions_dict = {"target": newdf.set_index('idx_test')['predicted_label'].to_dict()}

# Guardar el diccionario como un archivo JSON
import json

# Nombre del archivo JSON de salida
output_json_file = "predictions.json"

# Guardar el diccionario como un archivo JSON
with open(output_json_file, 'w') as json_file:
    json.dump(predictions_dict, json_file, indent = 4)
